In [2]:
# import numpy as np
# from numpy.polynomial.polynomial import polyfit
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression

In [6]:
import numpy as np
import pandas as pd
# import seaborn as sns

In [7]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from numpy.polynomial.polynomial import polyfit

AttributeError: module 'scipy' has no attribute '_lib'

In [8]:
from scipy import integrate

AttributeError: module 'scipy' has no attribute '_lib'

In [ ]:
df_transactions = pd.read_csv('transactions.csv')

In [ ]:
df_transactions.info()

## Seasonality
### Monthly


Calculate the revenue then find mean in each month

In [ ]:
df_transactions['revenue'] = df_transactions['unit_price'] * df_transactions['quantity']

In [ ]:
df_transactions['date'] = pd.to_datetime(df_transactions.date, format='%Y-%m-%d')

In [ ]:
df_transactions['month']= df_transactions['date'].dt.month

In [ ]:
df_transactions.head()

In [ ]:
Sum_rev_monthly=df_transactions.groupby(["month"]).sum()['revenue']

In [ ]:
print(Sum_rev_monthly)

In [ ]:
month = ['July','August','September','October','November','December']

plt.bar(month,Sum_rev_monthly)
plt.xlabel("Month")
plt.ylabel("Sum of revenue")
plt.title("Monthly revenue from July to December")

July - semester break, less student on campus
lower workload
August- start of semester
September 
October - mid-term+ assignment

### Day

In [ ]:
hours_open = 6

total_rev_each_day = df_transactions.groupby(["day_of_week", 'date']).sum()['revenue']
mean_daily_rev = total_rev_each_day.reset_index(level=0).groupby('day_of_week').mean()
mean_daily_rev_ph = mean_daily_rev/hours_open

mean_daily_rev_ph

In [ ]:
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

plt.bar(days, mean_daily_rev_ph.loc[days]['revenue'])
plt.xlabel('Days')
plt.ylabel('Revenue')
plt.title('Average daily revenue')

### Hourly

In [ ]:
Sum_rev_hourly=df_transactions.groupby(["hours_after_open"]).sum()['revenue']

Not sure if can change limit

In [ ]:
Hour = ['7-8','8-9','9-10','10-11','11-12','12-1']

plt.bar(Hour,Sum_rev_hourly)
plt.ylim((20000,28000)) 
plt.xlabel("Hour")
plt.ylabel("Sum of revenue")
plt.title("Hourly revenue from 7pm to 1am")

## Days Since Opened

1. Check out the skewness

In [ ]:
Transdays = df_transactions[["days_after_open"]]

In [ ]:
Transdays.hist(grid=False,
       figsize=(10, 6),
       bins=30)
       
plt.xlabel('Revenue')
plt.ylabel('Transaction value')
plt.title('Transaction Hist')
plt.show()

In [ ]:
Transdays.agg(['skew', 'kurtosis']).transpose()

In [ ]:
x = df_transactions["days_after_open"].to_numpy()
y = df_transactions["revenue"].to_numpy()

2.Try out for linear regression

In [ ]:
Daily_rev =df_transactions.groupby(["days_after_open"]).sum()['revenue']

In [ ]:
numbers = df_transactions["days_after_open"]

def get_unique_numbers(numbers):
    list_of_unique_numbers = []
    unique_numbers = set(numbers)
    for number in unique_numbers:
        list_of_unique_numbers.append(number)
    return list_of_unique_numbers

Unique = get_unique_numbers(numbers)

In [ ]:
Unique_ints = []
for element in Unique:
    Unique_ints.append(int(element))
print(Unique_ints)

In [ ]:
plt.scatter(Unique,Daily_rev,s=10,alpha = 0.5)

coef = np.polyfit(Unique,Daily_rev,1)
poly1d_fn = np.poly1d(coef) 

plt.plot(Unique, poly1d_fn(Unique), '--',color='r')
# Cannot plot linear line? since there are no linear relationship

plt.xlabel('Days since opened')
plt.ylabel('Revenue')
plt.title('Revenue of days since opened')
plt.savefig("plot")

## Transform the data

In [ ]:
x = df_transactions ['days_after_open'].values.reshape(-1, 1)
y = df_transactions ['revenue'].values

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x = sc_X.fit_transform(x)

In [ ]:
corr = np.corrcoef(x[:,0],y)

In [ ]:
corr

In [ ]:
plt.figure(figsize=(4,3))
x = sns.heatmap(corr,annot=True,annot_kws={"size": 16})
x.set_yticklabels(x.get_ymajorticklabels(), fontsize = 16)
x.set_xticklabels(x.get_xmajorticklabels(), fontsize = 16)
plt.show()

## Customer Location

In [ ]:
Rain = pd.get_dummies(df_transactions['raining'],drop_first = True)

print(Rain)

In [ ]:
Weather = df_transactions.groupby(["raining"]).sum()[['revenue','quantity']]

Weather

### Update method

In [ ]:
daily_location_rev = df_transactions.groupby(["study_area_name"]).count()[['raining']]
mean_daily_location_rev1 = daily_location_rev.reset_index(level=0).groupby('study_area_name').mean()

mean_daily_location_rev1

In [ ]:
daily_location_rev = df_transactions.groupby(["study_area_name"]).sum()[['revenue','quantity']]
mean_daily_location_rev2 = daily_location_rev.reset_index(level=0).groupby('study_area_name').mean()

mean_daily_location_rev2

In [ ]:
mean_daily_location_rev2['raining'] = mean_daily_location_rev1['raining']

In [ ]:
mean_daily_location_rev2

In [ ]:
plt.barh(mean_daily_location_rev2.index, mean_daily_location_rev2['revenue'],label='revenue')
plt.barh(mean_daily_location_rev2.index, mean_daily_location_rev2['quantity'],label='quantity')
plt.barh(mean_daily_location_rev2.index, mean_daily_location_rev2['raining'],label='raining')
plt.legend()
plt.show()

# Task 2 

In [ ]:
df = df_transactions[['name','day_of_week','unit_price','raining','hours_after_open','quantity']]

In [ ]:
df = pd.get_dummies(df, columns = ['name','day_of_week','raining'])

In [ ]:
x = df.drop('quantity',axis = 1)
y = df['quantity']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

In [ ]:
y_pred = lin_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False) # MSE

In [ ]:
mean_squared_error(y_test, y_pred, squared=True) # RMSE

In [ ]:
lin_reg.coef_